In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from itertools import product
sns.set()
font_titlesize = 18
font_axeslabelsize = 14
font_legendsize = 10

In [2]:
# Loading data
df = pd.read_csv('movieReplicationSet.csv')
movie_col_count = 400
df.head()

,The Life of David Gale (2003),Wing Commander (1999),Django Unchained (2012),Alien (1979),Indiana Jones and the Last Crusade (1989),Snatch (2000),Rambo: First Blood Part II (1985),Fargo (1996),Let the Right One In (2008),Black Swan (2010),...,When watching a movie I cheer or shout or talk or curse at the screen,When watching a movie I feel like the things on the screen are happening to me,As a movie unfolds I start to have problems keeping track of events that happened earlier,"The emotions on the screen ""rub off"" on me - for instance if something sad is happening I get sad or if something frightening is happening I get scared",When watching a movie I get completely immersed in the alternative reality of the film,Movies change my position on social economic or political issues,When watching movies things get so intense that I have to stop watching,Gender identity (1 = female; 2 = male; 3 = self-described),Are you an only child? (1: Yes; 0: No; -1: Did not respond),Movies are best enjoyed alone (1: Yes; 0: No; -1: Did not respond)
0,NaN,NaN,4.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,1.0,6.0,2.0,5.0,5.0,5.0,1.0,1.0,0,1
1,NaN,NaN,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,1.0,1.0,6.0,5.0,3.0,2.0,1.0,0,0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,4.0,3.0,5.0,5.0,4.0,4.0,1.0,1,0
3,NaN,NaN,2.0,NaN,3.0,NaN,NaN,NaN,NaN,4.0,...,3.0,1.0,1.0,4.0,5.0,3.0,1.0,1.0,0,1
4,NaN,NaN,3.5,NaN,0.5,NaN,0.5,1.0,NaN,0.0,...,2.0,3.0,2.0,5.0,6.0,4.0,4.0,1.0,1,1


In [3]:
movie_df = df.iloc[:, 0:movie_col_count+1]
movie_array = movie_df.to_numpy()
replaced_values = np.zeros(np.shape(movie_array))
nan_check = np.isnan(movie_array)

for ii, jj in product(range(np.shape(nan_check)[0]), range(np.shape(nan_check)[1])):
    if nan_check[ii, jj] == 1:
        replaced_values[ii, jj] = round((np.nanmean(df.iloc[ii, :]) + np.nanmean(df.iloc[:, jj]))/2, 1)
for ll in range(movie_col_count+1):
    df.iloc[:, ll] = replaced_values[:, ll]
df.head()

,The Life of David Gale (2003),Wing Commander (1999),Django Unchained (2012),Alien (1979),Indiana Jones and the Last Crusade (1989),Snatch (2000),Rambo: First Blood Part II (1985),Fargo (1996),Let the Right One In (2008),Black Swan (2010),...,When watching a movie I cheer or shout or talk or curse at the screen,When watching a movie I feel like the things on the screen are happening to me,As a movie unfolds I start to have problems keeping track of events that happened earlier,"The emotions on the screen ""rub off"" on me - for instance if something sad is happening I get sad or if something frightening is happening I get scared",When watching a movie I get completely immersed in the alternative reality of the film,Movies change my position on social economic or political issues,When watching movies things get so intense that I have to stop watching,Gender identity (1 = female; 2 = male; 3 = self-described),Are you an only child? (1: Yes; 0: No; -1: Did not respond),Movies are best enjoyed alone (1: Yes; 0: No; -1: Did not respond)
0,2.5,2.5,0.0,2.8,0.0,2.8,2.6,2.9,2.7,2.9,...,1.0,6.0,2.0,5.0,5.0,5.0,1.0,1.0,0,1
1,2.5,2.5,0.0,2.8,2.8,2.8,2.6,2.9,2.7,2.9,...,3.0,1.0,1.0,6.0,5.0,3.0,2.0,1.0,0,0
2,2.6,2.6,3.1,2.9,2.9,2.9,2.7,3.0,2.8,3.0,...,5.0,4.0,3.0,5.0,5.0,4.0,4.0,1.0,1,0
3,2.5,2.4,0.0,2.7,0.0,2.7,2.6,2.8,2.6,0.0,...,3.0,1.0,1.0,4.0,5.0,3.0,1.0,1.0,0,1
4,2.4,2.3,0.0,2.7,0.0,2.6,0.0,0.0,2.6,0.0,...,2.0,3.0,2.0,5.0,6.0,4.0,4.0,1.0,1,1


#### 1) For each of the 400 movies, use a simple linear regression model to predict the ratings. Use the ratings of the *other* 399 movies in the dataset to predict the ratings of each movie (that means you’ll have to build 399 models for each of the 400 movies). For each of the 400 movies, find the movie that predicts ratings the best. Then report the average COD of those 400 simple linear regression models. Please include a histogram of these 400 COD values and a table with the 10 movies that are most easily predicted from the ratings of a single other movie and the 10 movies that are hardest to predictfrom the ratings of a single other movie (and their associated COD values, as well as which movie ratings are the best predictor, so this table should have 3 columns).

In [20]:
movie_df = df.iloc[:, 0:movie_col_count+1]
COD_array = np.zeros(movie_col_count+1,)
highest_rated_movie = np.zeros(movie_col_count,)
for ii in range(movie_col_count+1):
    X = movie_df.drop(movie_df.columns[ii], axis = 1)
    y = movie_df.iloc[:, ii]
    reg = LinearRegression().fit(X, y)
    betas = reg.coef_
    highest_rated_movie[ii] = np.argmax(abs(betas))
    COD_array[ii] = reg.score(X, y)

IndexError: index 400 is out of bounds for axis 0 with size 400

In [17]:
plt.figure()
plt.hist(COD_array)
plt.show()

[[1 1]
 [1 2]
 [2 2]
 [2 3]]
[ 6  8  9 11]
[1. 2.]
3.000000000000001


#### 2) For the 10 movies that are best and least well predicted from the ratings of a single other movie (so 20 in total), build multiple regression models that include gender identity (column 475), sibship status (column 476) and social viewing preferences (column 477) as additional predictors (in addition to the best predicting movie from question 1). Comment on how R^2 has changed relative to the answers in question 1. Please include a figure with a scatterplot where the old COD (for the simple linear regression models from the previous question) is on the x-axis and the new R^2 (for the new multiple regression models) is on the y-axis.

#### 3) Pick 30 movies in the middle of the COD range, as identified by question 1 (that were not used in question 2). Now build a regularized regression model with the ratings from 10 other movies (picked randomly, or deliberately by you) as an input. Please use ridge regression, and make sure to do suitable hyperparameter tuning. Also make sure to report the RMSE for each of these 30 movies in a table, after doing an 80/20 train/test split. Comment on the hyperparameters you use and betas you find by doing so.

#### 4) Repeat question 3) with LASSO regression. Again, make sure to comment on the hyperparameters you use and betas you find by doing so. 

#### 5) Compute the average movie enjoyment for each user (using only real, non-imputed data). Use these averages as the predictor variable X in a logistic regression model. Sort the movies order of increasing rating (also using only real, non-imputed data). Now pick the 4 movies in the middle of the score range as your target movie. For each of them, do a media split (now using the imputed data) of ratings to code movies above the median rating with the Y label 1 (= enjoyed) and movies below the median with the label 0 (= not enjoyed). For each of these movies, build a logistic regression model (using X to predict Y), show figures with the outcomes and report the betas as well as the AUC values. Comment on the quality of your models. Make sure to use cross-validation methods to avoid overfitting.